In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from pythainlp.tokenize import word_tokenize
from string import punctuation

import numpy as np

import time

def reset(df):
    cols = df.columns
    return df.reset_index()[cols]

print('OK !')

OK !


In [4]:
def canonicalize(string):
    normalized_tokens = list()
    a = word_tokenize(string, engine = 'newmm')
    for j in a:
        token = j.strip()
        #Add clean statement here 
        if len(token) > 1 and token not in set(punctuation) and token not in ['..','...','ๆๆ']:
            try:
                normalized_tokens.append(token.lower())
            except:
                normalized_tokens.append(token)
                pass
    return normalized_tokens

def query(query_text, vectorizer, document_matrix, df, searched_field):
    
    # Transform our string using the vocabulary
    transformed = vectorizer.transform([query_text])
    query = transformed[0:1]

    np_result = np.array(find_similar(query, document_matrix, len(df)))

    mask = np_result[:, 1] > 0 

    np_result = np_result[mask, :]
        
    index_list = []
    score_list = []
    df_dict = {'index': index_list, 'score': score_list}

    df_dict['index'] =  np_result[:,0]
    df_dict['score'] =  np_result[:,1]

    df = pd.DataFrame(df_dict)
    df['index'] = df['index'].astype(int)
    df = df.set_index('index')
    
    return reset(pd.merge(df, data, left_index=True, right_index=True, how ='left'))


# A function that given an input query item returns the top-k most similar items 
# by their cosine similarity.
def find_similar(query_vector, td_matrix, top_k = 5):
    cosine_similarities = cosine_similarity(query_vector, td_matrix).flatten()
    related_doc_indices = cosine_similarities.argsort()[::-1]
    return [(index, cosine_similarities[index]) for index in related_doc_indices][0:top_k]

print('OK !')

OK !


In [5]:
# We pass in our tokenizer to the vectorizer object.

tfidf_vectorizer = TfidfVectorizer(tokenizer=canonicalize, sublinear_tf=True)

print('OK')

OK


In [6]:
data = pd.read_csv('data/clean_sample_text.csv')

data.head()

,published,content,engagement,len
0,11/07/20 00:09,โดน​ค่า ATS 14\r\n บาท​กับ​หุ้น​อยู่​เลย,0.0,38
1,11/07/20 00:51,ถ้า​ตาม​นิยาม​นี้\r\n การ​ผูก​บัตร​เครดิต​กับ...,0.0,184
2,11/07/20 01:30,ไร้​เงินสด เครดิต​ก็​ไม่​มี\r\n หนี้​นอก​ระบบ...,0.0,60
3,11/07/20 01:34,ถั่ว​ต้ม​เลย ไร้​เงินสด\r\n ไร้​ของ​จะ​กิน,0.0,40
4,11/07/20 03:30,อยากให้ลดข้อแม้ในการเข้าถึงแหล่งเงินทุนของสถาบ...,0.0,278


In [7]:
#This can take time; after finnishing this, you can save transformed document into pickle file for later usage
start = time.time()

tfidf_term_document_matrix = tfidf_vectorizer.fit_transform(data['content'])

print("Took " + "%.2f" % float(time.time() - start) + " sec")

Took 1.04 sec


In [8]:
query_text = "สนุก"
searched_field = 'content'

result_list = query(query_text, tfidf_vectorizer, tfidf_term_document_matrix, data, searched_field)

result_list

,score,published,content,engagement,len
0,0.439635,18/08/20 16:03,พวก​หัว​สาย​เอา​เงิน​ไป​จ่าย​กัน​สนุก​มา​นาน​แล้ว,0.0,49
1,0.360820,13/07/20 20:31,อัน​นี้​สนุก​แน่​ครับ\r\n https://www.faceboo...,0.0,87
2,0.324446,13/07/20 20:35,อัน​นี้​มี​สนุก​และ​สี​สัน​แน่นอน\r\n https:/...,0.0,99


In [9]:
query_text = "เที่ยว"
searched_field = 'content'

result_list = query(query_text, tfidf_vectorizer, tfidf_term_document_matrix, data, searched_field)

result_list

,score,published,content,engagement,len
0,0.513902,17/08/20 17:35,ไป​เที่ยว​ไหน​น​น​น​น,0.0,21
1,0.444235,17/08/20 11:53,Sitanan Chuanoon เที่ยว​อีก​แล้ว,0.0,32
2,0.341085,17/08/20 11:36,Sageenah Nae มาก\r\n เที่ยว​ไหน​ล่ะ​หล่อน,0.0,39
3,0.336529,17/08/20 13:07,Aomm Thanomsub เที่ยว​สกล​จ้ะ​พี่,0.0,33
4,0.332468,17/08/20 20:48,Chanut Nut เเท็ก​ไมจะให้​ไป​เที่ยว​บ้าน​หรอ​5555,0.0,48
5,0.329796,17/08/20 13:57,ไป​เที่ยว​กัน​ไหม​เจ้า​เหมียว\r\n Titichaya T...,0.0,49
6,0.323319,17/08/20 11:14,เฟริส สี๋สี๋\r\n พา​ไป​เที่ยว​หน่อยยยยยย,0.0,38
7,0.314003,17/08/20 14:26,Aumarin Shuengharuethai Cha'may\r\n Begin ไป​...,0.0,53
8,0.313257,17/08/20 22:06,Mas Mas เที่ยว​ไหนดี ตอบมมสิ😁,0.0,29
9,0.311895,17/08/20 13:19,Thong Krubpom ปายยยยยยยยยยยยยยยยยยยยเที่ยว​ว​ว...,0.0,62


In [10]:
word2tfidf = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer.idf_))
sorted_dict = {k: v for k, v in sorted(word2tfidf.items(), key=lambda item: item[1], reverse=True)}

In [11]:
for word, score in sorted_dict.items():
    print( "'" + word + "'", score)

'!!!!' 8.287902579106012
'!!!!!!!!!!!!' 8.287902579106012
'!?!' 8.287902579106012
'"...' 8.287902579106012
'"ขอ​ลด​' 8.287902579106012
'"ยก' 8.287902579106012
'"ลด​' 8.287902579106012
'"✨' 8.287902579106012
'###################' 8.287902579106012
'#กรณ์' 8.287902579106012
'#กลต' 8.287902579106012
'#ขอ' 8.287902579106012
'#คง' 8.287902579106012
'#คปภ' 8.287902579106012
'#ครนเบ' 8.287902579106012
'#ผม' 8.287902579106012
'#มอร์มูฟ' 8.287902579106012
'#ยก' 8.287902579106012
'#ลด' 8.287902579106012
'#ศคง' 8.287902579106012
'#ส' 8.287902579106012
'#สรท' 8.287902579106012
'#สส' 8.287902579106012
'$$$' 8.287902579106012
'%).' 8.287902579106012
'%)......' 8.287902579106012
'%++)' 8.287902579106012
''สห' 8.287902579106012
'(+)' 8.287902579106012
'(กน​ง.)' 8.287902579106012
'(ณ' 8.287902579106012
'(ตบ​' 8.287902579106012
'(ธ' 8.287902579106012
'(บสย.)' 8.287902579106012
'(ปอท.)' 8.287902579106012
'(สรท.)' 8.287902579106012
'(สอ' 8.287902579106012
')(' 8.287902579106012
'),' 8.287902579106012
').'

'phakpum' 8.287902579106012
'phatcharida' 8.287902579106012
'phe' 8.287902579106012
'phenkhatwid' 8.287902579106012
'phet' 8.287902579106012
'phetcharat' 8.287902579106012
'pheung' 8.287902579106012
'pheungprincess' 8.287902579106012
'phitchakorn' 8.287902579106012
'phoklarp' 8.287902579106012
'phoklin' 8.287902579106012
'phone' 8.287902579106012
'phongdanai' 8.287902579106012
'phongthaninthorn' 8.287902579106012
'phoyen' 8.287902579106012
'phruettiphat' 8.287902579106012
'phumichat' 8.287902579106012
'phurahong' 8.287902579106012
'piamsaad' 8.287902579106012
'piano' 8.287902579106012
'pichakorn' 8.287902579106012
'pichapa' 8.287902579106012
'pichet' 8.287902579106012
'pichetpakorn' 8.287902579106012
'pichetsak' 8.287902579106012
'pier' 8.287902579106012
'pila' 8.287902579106012
'pimdee' 8.287902579106012
'pimlapat' 8.287902579106012
'pimpakan' 8.287902579106012
'pimpika' 8.287902579106012
'pimpisa' 8.287902579106012
'pimvalee' 8.287902579106012
'ping' 8.287902579106012
'pink' 8.287902

'ทุนสำรอง' 8.287902579106012
'ทุนสำรองเงินตรา' 8.287902579106012
'ทุร' 8.287902579106012
'ทุ่ง' 8.287902579106012
'ทุ่ม' 8.287902579106012
'ทู' 8.287902579106012
'ทูต' 8.287902579106012
'ทูบ' 8.287902579106012
'ทูเดย์' 8.287902579106012
'ทแยง' 8.287902579106012
'ท่า' 8.287902579106012
'ท้อ' 8.287902579106012
'ท้อใจ' 8.287902579106012
'ท้า' 8.287902579106012
'ท​' 8.287902579106012
'ท​วง​' 8.287902579106012
'ธง' 8.287902579106012
'ธน' 8.287902579106012
'ธนัช' 8.287902579106012
'ธนาคารเพื่อการส่งออกและนำเข้าแห่งประเทศไทย' 8.287902579106012
'ธพ' 8.287902579106012
'ธรรมดา' 8.287902579106012
'ธรรมนูญ' 8.287902579106012
'ธรรมศาสตร์' 8.287902579106012
'ธรรมาภิบาล' 8.287902579106012
'ธอร์' 8.287902579106012
'ธัมโม' 8.287902579106012
'ธาดา' 8.287902579106012
'ธำรง' 8.287902579106012
'ธิดา' 8.287902579106012
'ธิ์' 8.287902579106012
'ธีระ' 8.287902579106012
'ธุ' 8.287902579106012
'ธุรกิจขนาดเล็ก' 8.287902579106012
'ธเนศ' 8.287902579106012
'ธ​ปม' 8.287902579106012
'ธ​ป​ค.' 8.287902579106012
'น'' 8.

'ให้อำนาจ' 8.287902579106012
'ให้แรง' 8.287902579106012
'ให้โอกาส' 8.287902579106012
'ใาก' 8.287902579106012
'ไก่' 8.287902579106012
'ไข่ไก่' 8.287902579106012
'ไฉน' 8.287902579106012
'ไช' 8.287902579106012
'ได่' 8.287902579106012
'ได้กัน' 8.287902579106012
'ได้ขนาด' 8.287902579106012
'ได้คิด' 8.287902579106012
'ไต้หวัน' 8.287902579106012
'ไถล' 8.287902579106012
'ไทยยาม' 8.287902579106012
'ไทยรัฐ' 8.287902579106012
'ไทยแลนด์' 8.287902579106012
'ไทยโพสต์' 8.287902579106012
'ไน' 8.287902579106012
'ไบเดน' 8.287902579106012
'ไปนอก' 8.287902579106012
'ไปเที่ยว' 8.287902579106012
'ไผ่' 8.287902579106012
'ไพ' 8.287902579106012
'ไพรินทร์' 8.287902579106012
'ไฟฉาย' 8.287902579106012
'ไฟฟ้า' 8.287902579106012
'ไฟล์' 8.287902579106012
'ไฟเขียว' 8.287902579106012
'ไม' 8.287902579106012
'ไมค์' 8.287902579106012
'ไมตรี' 8.287902579106012
'ไมเนอร์' 8.287902579106012
'ไม่งั้น' 8.287902579106012
'ไม่ต้องการ' 8.287902579106012
'ไม่ยังงั้น' 8.287902579106012
'ไม่อย่างนั้น' 8.287902579106012
'ไม่เบา' 8.28

'ภายในประเทศ' 7.882437470997847
'ภารกิจ' 7.882437470997847
'ภูมิ' 7.882437470997847
'ภูมิภาค' 7.882437470997847
'ภูริ' 7.882437470997847
'ภูเขา' 7.882437470997847
'มลพิษ' 7.882437470997847
'มสิ' 7.882437470997847
'มหาวิทยาลัย' 7.882437470997847
'มอ' 7.882437470997847
'มองว่า' 7.882437470997847
'มองโลก' 7.882437470997847
'มะเร็ง' 7.882437470997847
'มักจะ' 7.882437470997847
'มัง' 7.882437470997847
'มันดี' 7.882437470997847
'มั๊ย' 7.882437470997847
'มากยิ่งขึ้น' 7.882437470997847
'มาตรฐานสากล' 7.882437470997847
'มาบ' 7.882437470997847
'มารุต' 7.882437470997847
'มาเลเซีย' 7.882437470997847
'มิค' 7.882437470997847
'มิตร' 7.882437470997847
'มีด' 7.882437470997847
'มีนาคม' 7.882437470997847
'มีบทบาท' 7.882437470997847
'มีผลต่อ' 7.882437470997847
'มีวินัย' 7.882437470997847
'มือสอง' 7.882437470997847
'มุ' 7.882437470997847
'มุ่งมั่น' 7.882437470997847
'มุ่งหวัง' 7.882437470997847
'มู' 7.882437470997847
'มูล' 7.882437470997847
'มโน' 7.882437470997847
'ม่าย' 7.882437470997847
'ยยย' 7.88243747099

'ใช้หนี้' 7.371611847231857
'ในขณะที่' 7.371611847231857
'ไกล' 7.371611847231857
'ได้ประโยชน์' 7.371611847231857
'ได้ยิน' 7.371611847231857
'ได้รับความเดือดร้อน' 7.371611847231857
'ได้เงิน' 7.371611847231857
'ไปดู' 7.371611847231857
'ไวรัส' 7.371611847231857
'ไห้' 7.371611847231857
'​ง.' 7.371611847231857
'​ชน​' 7.371611847231857
'​บ' 7.371611847231857
'​บน​' 7.371611847231857
'​ป' 7.371611847231857
'​ป​ช​ช.' 7.371611847231857
'​ล' 7.371611847231857
'​อ' 7.371611847231857
'​อก​' 7.371611847231857
'​อ​ยุ่' 7.371611847231857
'​เปน' 7.371611847231857
'⁉️' 7.371611847231857
'✨🔎' 7.371611847231857
'%.' 7.189290290437902
'.....' 7.189290290437902
'1.3' 7.189290290437902
'1.5' 7.189290290437902
'14' 7.189290290437902
'16.30' 7.189290290437902
'40' 7.189290290437902
'5,000' 7.189290290437902
'5500.' 7.189290290437902
'737' 7.189290290437902
':)' 7.189290290437902
'???' 7.189290290437902
'access' 7.189290290437902
'anthony' 7.189290290437902
'anywhere' 7.189290290437902
'assure' 7.1892902904379

'แก้' 4.76154205448985
'​ง​ค์' 4.76154205448985
'ได้ที่' 4.746943255068698
'ความ' 4.732554517616598
'ค่า' 4.732554517616598
'ติดตาม' 4.732554517616598
'หลาย' 4.732554517616598
'ระบบ' 4.718369882624641
'รับ' 4.718369882624641
'ขอบคุณ' 4.704383640649901
'ดู' 4.704383640649901
'ลูกหนี้' 4.704383640649901
'เขา' 4.704383640649901
'ไว้' 4.704383640649901
'ปรับ' 4.6905903185175655
'กู้' 4.676984666461787
'ธุรกิจ' 4.676984666461787
'ก่อน' 4.663561646129646
'นโยบาย' 4.650316419379626
'อยาก' 4.650316419379626
'​คน​' 4.650316419379626
'ไหน' 4.637244337812273
'คิด' 4.624340932976365
'ประชาชน' 4.624340932976365
'ผ่าน' 4.624340932976365
'สินเชื่อ' 4.611601907198935
'แบบ' 4.611601907198935
'บาท' 4.599023124992075
'อ่าน' 4.599023124992075
'ทาง' 4.586600604993518
'ออก' 4.586600604993518
'อะไร' 4.538398503175641
'งาน' 4.526702463412449
'เดือน' 4.526702463412449
'เอา' 4.526702463412449
'การเงิน' 4.515141641011374
'ทั้ง' 4.515141641011374
'แบ' 4.5037129451877504
'จ่าย' 4.492413389933818
'นาน' 4.4924133899

In [12]:
tfidf_vectorizer.vocabulary_

{'โดน': 8057,
 'ค่า': 3726,
 'ats': 578,
 '14': 169,
 'บาท': 4965,
 'กับ': 3175,
 'หุ้น': 6771,
 'อยู่': 6823,
 'เลย': 7581,
 'ถ้า': 4434,
 'ตาม': 4284,
 'นิยาม': 4831,
 'นี้': 4840,
 'การ': 3183,
 'ผูก': 5265,
 'บัตร': 4948,
 'เครดิต': 7106,
 'แอ': 8010,
 '\u200bพ': 8519,
 'ถือว่า': 4414,
 'เป็นการ': 7395,
 'เปลี่ยน': 7373,
 'instrument': 1267,
 'จาก': 3851,
 'card': 724,
 'เป็น': 7393,
 'mobile': 1682,
 'banking': 602,
 'แม้': 7952,
 'ว่า': 6255,
 'จะ': 3828,
 'จ่าย': 3895,
 'แบบ': 7902,
 'credit': 883,
 'เหมือน': 7701,
 'เดิม': 7251,
 'ถูก': 4420,
 'ต้อง': 4372,
 'มั้ย': 5621,
 'ครับ': 3539,
 'ไร้': 8304,
 'เงินสด': 7151,
 'ก็': 3334,
 'ไม่': 8289,
 'มี': 5656,
 'หนี้': 6621,
 'นอก': 4752,
 'ระบบ': 5884,
 'ดอก': 4079,
 'ร้อย': 6040,
 'ละ': 6086,
 'ถั่ว': 4400,
 'ต้ม': 4369,
 'ของ': 3384,
 'กิน': 3307,
 'อยาก': 6820,
 'ให้': 8215,
 'ลด': 6069,
 'ข้อแม้': 3474,
 'ใน': 8187,
 'เข้าถึง': 7086,
 'แหล่งเงินทุน': 8003,
 'สถาบันการเงิน': 6324,
 'ต่างๆ': 4355,
 'บ้าง': 5008,
 'หนี้เสีย': 662